In [40]:
import pandas as pd
import re
import json
import progressbar
import csv
from collections import Counter, OrderedDict

In [2]:
df = pd.read_csv(r"C:\Users\kmoudgalya\Desktop\AI PRD\Sample\Sample.csv")

In [3]:
df_final = df["_raw"]

In [4]:
df_final_copy = df_final

In [5]:
df_final = df_final.dropna()

In [6]:
df_final.shape

(25077,)

In [7]:
df.columns[0]

'_raw'

# Get the Json type logs from the sample.csv

In [8]:
found_json = {}
#regex_timestamp = r"[0-9]+:[0-9]+:[0-9]+.[0-9]+|[0-9]+:[0-9]+:[0-9]+:[0-9]+|[0-9]+:[0-9]+:[0-9]+"
#regex_timestamp = r"[0-9]{2}:[0-9]{2}:[0-9]{2}.[0-9]{6}|[0-9]{2}:[0-9]{2}:[0-9]{2}:[0-9]{6}|[0-9]{2}:[0-9]{2}:[0-9]{2}"

#if the regex becomes too long, put them in different strings and concatenate them in the end
for key, value in df_final.iteritems():
    #matches = re.findall(regex_timestamp, str(value))
    if value.startswith("{") and value.endswith("}"):
        found_json[key] = json.loads(value)

In [9]:
found_json

{90: {'bytes': 2463,
  'bytes_in': 735,
  'bytes_out': 1728,
  'dest_ip': '10.2.1.133',
  'dest_port': 443,
  'http_method': 'GET',
  'http_user_agent': 'Java/Buttercup',
  'site': 'api.buttercup.com:443',
  'src_ip': '23.20.53.173',
  'status': 200,
  'time_taken': 179,
  'timestamp': '2018-10-23 15:02:19.115992',
  'transport': 'tcp',
  'url': '/inventory/search?q=Man'},
 91: {'bytes': 2952,
  'bytes_in': 450,
  'bytes_out': 2502,
  'dest_content': '{"orderstatus": "received"}',
  'dest_ip': '10.2.1.135',
  'dest_port': 443,
  'http_method': 'POST',
  'http_user_agent': 'Java/Buttercup',
  'site': 'api.buttercup.com:443',
  'src_content': '{"orderid": "ED9FDFFDFFFE"}',
  'src_ip': '23.20.53.146',
  'status': 200,
  'time_taken': 190,
  'timestamp': '2018-10-23 15:02:19.115981',
  'transport': 'tcp',
  'url': '/user/order'},
 92: {'bytes': 3259,
  'bytes_in': 420,
  'bytes_out': 2839,
  'dest_ip': '10.2.1.133',
  'dest_port': 443,
  'http_method': 'POST',
  'http_user_agent': 'Java/Bu

# Get the set of keys inside the json data

In [10]:
column_list_json = []
for key in found_json:
    column_list_json.append(list(found_json[key].keys()))

# Create a list of keys across all the json data

In [11]:
flat_column_list_json = []
for sublist in column_list_json:
    for item in sublist:
        flat_column_list_json.append(item)

In [12]:
len(flat_column_list_json)

43465

In [13]:
len(set(flat_column_list_json))

283

# Get unique set of keys, which will be the column names of the dataframe

In [14]:
unique_json_columns = set(flat_column_list_json)

In [15]:
for key in found_json:
    if len(list(found_json[key].keys())) != len(set(list(found_json[key].keys()))):
        print(key)

# Create a dataframe with all the necessary columns

In [16]:
rows = list(found_json.keys())
cols = unique_json_columns
df = pd.DataFrame(([None for col in cols] for row in rows),
                 index = rows, columns=cols)

In [17]:
df.head()

,src_content,maxTime,input_name,jvmDescription,is_entity_in_maintenance,application_summary.throughput,orig_sourcetype,responsetime,thresholdresponsetimecritical,itsi_first_event_id,...,sys_domain,secureRandomClass,drilldown_uri,itsi_dummy_closing_flag,itsi_is_last_event,offset,Risk_Level,remoteIP,maxActive,requestCount
90,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
91,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
92,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
93,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
94,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


# This would take some time (3-5 mins)

In [18]:
pbar = progressbar.ProgressBar()
for key in pbar(found_json):
    for col in cols:
        if col in list(found_json[key].keys()):
            df.loc[[key],[col]] = found_json[key][col]

100% (2474 of 2474) |####################| Elapsed Time: 0:10:12 Time:  0:10:12


In [19]:
df

,src_content,maxTime,input_name,jvmDescription,is_entity_in_maintenance,application_summary.throughput,orig_sourcetype,responsetime,thresholdresponsetimecritical,itsi_first_event_id,...,sys_domain,secureRandomClass,drilldown_uri,itsi_dummy_closing_flag,itsi_is_last_event,offset,Risk_Level,remoteIP,maxActive,requestCount
90,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
91,"{""orderid"": ""ED9FDFFDFFFE""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
92,"{""userid"": ""rroberts1""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
93,"{""userid"": ""wrose8""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
94,"{""userid"": ""abishopz""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
95,"{""userid"": ""gprice0""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
96,"{""cartid"": ""171FFFCEFD61"", ""item"": ""MCB-6""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
97,"{""cartid"": ""F6EEDDA73FED""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
98,"{""cartid"": ""FFDCF854DF06"", ""item"": ""PP-5""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
99,"{""orderid"": ""BFFEEAF18FFF""}",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [20]:
df.to_csv(r"C:\Users\kmoudgalya\Desktop\AI PRD\Sample\json_data.csv")

In [21]:
# from collections import Counter, OrderedDict
# sorted(Counter(df).items(), key=lambda t: t[1], reverse = True)
# print(Counter(found_keys_new).keys())
# print(Counter(found_keys_new).values())
# print(len(Counter(found_keys_new).keys()))
# print(len(Counter(found_keys_new).values()))

# Count in new:

In [25]:
cols = list(cols)

In [29]:
cols[0]

'src_content'

In [34]:
def get_number_of_rows(filename, header):
      with open(filename) as f:
         reader = csv.DictReader(f)
         return sum([bool(row[header]) for row in reader])

In [37]:
reader = csv.DictReader(open(r"C:\Users\kmoudgalya\Desktop\AI PRD\Sample\json_data.csv"))
count_dict = {}
pbar = progressbar.ProgressBar()
for i in pbar(range(0,len(cols))):
    count = get_number_of_rows(r"C:\Users\kmoudgalya\Desktop\AI PRD\Sample\json_data.csv",cols[i])
    count_dict[cols[i]] = count

100% (283 of 283) |######################| Elapsed Time: 0:01:30 Time:  0:01:30


In [41]:
OrderedDict(sorted(Counter(count_dict).items(), key=lambda t: t[1], reverse = True))

OrderedDict([('timestamp', 1119),
             ('transport', 776),
             ('dest_port', 776),
             ('dest_ip', 776),
             ('src_ip', 776),
             ('time_taken', 776),
             ('bytes', 776),
             ('name', 744),
             ('status', 703),
             ('id', 612),
             ('account_id', 612),
             ('input_name', 606),
             ('reporting', 606),
             ('health_status', 606),
             ('event_id', 578),
             ('url', 522),
             ('transaction_name', 518),
             ('bytes_in', 410),
             ('site', 410),
             ('http_user_agent', 410),
             ('http_method', 410),
             ('bytes_out', 410),
             ('user', 370),
             ('query', 366),
             ('application_server', 352),
             ('mbean_domain', 352),
             ('mbean_property_type', 352),
             ('itsi_first_event_id', 290),
             ('severity', 290),
             ('description', 290),


# Count old:

In [ ]:
found_keys = []

regex_keys = r"\"[a-zA-Z0-9._\\]+\":"
count = 0
for key, value in json_df.iteritems():
    matches = re.findall(regex_keys, str(value))
    for j in matches:
            found_keys.append(str(j)[1:])

In [ ]:
len(found_keys)

In [ ]:
len(set(found_keys))

# I think we might be missing a few columns in this list